In [1]:
print('hello')

hello


In [ ]:
# code to extract 15 minute data from Yahoo Finance
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def download_intraday_data(ticker, start_date, end_date, interval="15m", chunk_days=30):
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    df_all = []

    while start < end:
        chunk_end = min(start + timedelta(days=chunk_days), end)
        print(f"Fetching: {start.date()} to {chunk_end.date()}")

        df = yf.download(
            ticker,
            start=start.strftime('%Y-%m-%d'),
            end=chunk_end.strftime('%Y-%m-%d'),
            interval=interval,
            progress=False
        )
        
        if not df.empty:
            df_all.append(df)

        # Add a buffer of 1 minute to avoid overlapping rows
        start = chunk_end + timedelta(minutes=1)

    # Combine all chunks
    if df_all:
        final_df = pd.concat(df_all)
        final_df = final_df[~final_df.index.duplicated(keep='first')]  # Remove any duplicate rows
        return final_df
    else:
        print("No data returned.")
        return pd.DataFrame()

# Example usage:
df_spy_15m = download_intraday_data("SPY", start_date="2023-12-01", end_date="2024-04-01", interval="15m")
print(df_spy_15m.head())